In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda"

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 32, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 1, dropout = 0.2):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
        
        self.lineartest0 = nn.Linear(2, emsize)
        self.lineartest1 = nn.Linear(2, emsize)
        self.lineartest2 = nn.Linear(2, emsize)
        self.lineartest3 = nn.Linear(2 * emsize, emsize)
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 2]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def test_encode(self, src): #src must be [batchsize * nsamples, npoints = 5, 2]
        point_1 = self.lineartest1(src[:, 3, :] - 0.5) #want mean 0!!!!
        point_2 = self.lineartest1(src[:, 4, :] - 0.5)
        remaining = self.lineartest0(src[:, :3, :]) #[batchsize * nsamples, 3, emsize]
        point_1_message = self.lineartest1(src[:, 3, :] - 0.5).squeeze(1)
        point_2_message = self.lineartest2(src[:, 4, :] - 0.5).squeeze(1)
        point_1 = F.relu(torch.cat([point_1, point_2_message], dim = 1))
        point_2 = F.relu(torch.cat([point_2, point_1_message], dim = 1))
        point_1 = self.lineartest3(point_1)
        point_2 = self.lineartest3(point_2)
        src = torch.cat([torch.zeros_like(remaining.transpose(0, 1)), point_1.unsqueeze(0), point_2.unsqueeze(0)])
        return src #[npoints = 5, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 3], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 3, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 3, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 3, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 3, npoints - 3], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment():
    def reset(self, npoints, batchsize, nsamples):
        if npoints <= 3:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = batchsize * nsamples #so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = torch.rand([batchsize, npoints - 3, 2], device = device).unsqueeze(1).expand(-1, nsamples, -1, -1).reshape(self.batchsize, npoints - 3, 2)
        self.corner_points = torch.tensor([[0, 0], [2, 0], [0, 2]], dtype = torch.float, device = device)
        self.points = torch.cat([self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1), self.points], dim = -2) #[batchsize * nsamples, npoints, 2]
        self.points_mask = torch.cat([torch.ones([self.batchsize, 3], dtype = torch.bool, device = device), torch.zeros([self.batchsize, npoints - 3], dtype = torch.bool, device = device)], dim = 1)
        self.points_sequence = torch.empty([self.batchsize, 0], dtype = torch.long, device = device)
        
        """use a trick, for the purpose of an 'external' triangle that is always left untouched, which means we don't have to deal with boundary edges as being different. external triangle is [0, 1, 2] traversed clockwise..."""
        self.partial_delaunay_triangles = torch.tensor([[0, 2, 1], [0, 1, 2]], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1, -1).contiguous() #[batchsize, ntriangles, 3] contains index of points, always anticlockwise
        self.partial_delaunay_edges = torch.tensor([5, 4, 3, 2, 1, 0], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1).contiguous() #[batchsize, ntriangles * 3] contains location of corresponding edge (edges go in order 01, 12, 20). Edges will always flip since triangles are stored anticlockwise.
        
        self.ntriangles = 2 #can store as scalar, since will always be the same
        self.cost = torch.zeros([self.batchsize], device = device)
        
        self.logprob = torch.zeros([self.batchsize], device = device, requires_grad = True)
    
    def update(self, point_index): #point_index is [batchsize]
        if point_index.size(0) != self.batchsize:
            print("Error: point_index.size() doesn't match expected size, should be [batchsize]")
            return
        if self.points_mask.gather(1, point_index.unsqueeze(1)).sum():
            print("Error: some points already added")
            return
        triangles_coordinates = self.points.gather(1, self.partial_delaunay_triangles.view(self.batchsize, self.ntriangles * 3).unsqueeze(2).expand(-1, -1, 2)).view(self.batchsize, self.ntriangles, 3, 2) #[batchsize, ntriangles, 3, 2]
        newpoint = self.points.gather(1, point_index.unsqueeze(1).unsqueeze(2).expand(self.batchsize, 1, 2)).squeeze(1) #[batchsize, 2]
        
        incircle_matrix = torch.cat([triangles_coordinates, newpoint.unsqueeze(1).unsqueeze(2).expand(-1, self.ntriangles, 1, -1)], dim = -2) #[batchsize, ntriangles, 4, 2]
        incircle_matrix = torch.cat([incircle_matrix, (incircle_matrix * incircle_matrix).sum(-1, keepdim = True), torch.ones([self.batchsize, self.ntriangles, 4, 1], device = device)], dim = -1) #[batchsize, ntriangles, 4, 4]
        incircle_test = incircle_matrix.det() > 0 #[batchsize, ntriangles], is True if inside incircle
        removed_edge_mask = incircle_test.unsqueeze(2).expand(-1, -1, 3).reshape(-1) #[batchsize * ntriangles * 3]
        
        edges = (self.partial_delaunay_edges + self.ntriangles * 3 * torch.arange(self.batchsize, device = device).unsqueeze(1)).view(-1) #[batchsize * ntriangles * 3]
        neighbouring_edge = edges.masked_select(removed_edge_mask)
        neighbouring_edge_mask = torch.zeros([self.batchsize * self.ntriangles * 3], device = device, dtype = torch.bool)
        neighbouring_edge_mask[neighbouring_edge] = True
        neighbouring_edge_mask = (neighbouring_edge_mask * removed_edge_mask.logical_not()) #[batchsize * ntriangles * 3]
        
        n_new_triangles = neighbouring_edge_mask.view(self.batchsize, -1).sum(-1) #[batchsize]
        
        new_point = point_index.unsqueeze(1).expand(-1, self.ntriangles * 3).masked_select(neighbouring_edge_mask.view(self.batchsize, -1))
        
        second_point_mask = neighbouring_edge_mask.view(self.batchsize, -1, 3) #[batchsize, ntriangles 3]
        (first_point_indices0, first_point_indices1, first_point_indices2) = second_point_mask.nonzero(as_tuple = True)
        first_point_indices2 = (first_point_indices2 != 2) * (first_point_indices2 + 1)
        
        first_point = self.partial_delaunay_triangles[first_point_indices0, first_point_indices1, first_point_indices2] #[?]
        second_point = self.partial_delaunay_triangles.masked_select(second_point_mask) #[?]
        
        new_triangles_mask = torch.cat([incircle_test, torch.ones([self.batchsize, 2], dtype = torch.bool, device = device)], dim = 1) #[batchsize, ntriangles + 2]
        
        new_neighbouring_edges = 3 * new_triangles_mask.nonzero(as_tuple = True)[1] #[?], 3* since is the 01 edge of new triangles (see later)
        self.partial_delaunay_edges.masked_scatter_(neighbouring_edge_mask.view(self.batchsize, -1), new_neighbouring_edges) #still [batchsize, ntriangles * 3] for now
        
        self.partial_delaunay_triangles = torch.cat([self.partial_delaunay_triangles, torch.empty([self.batchsize, 2, 3], dtype = torch.long, device = device)], dim = 1)
        self.partial_delaunay_edges = torch.cat([self.partial_delaunay_edges, torch.empty([self.batchsize, 6], dtype = torch.long, device = device)], dim = 1)
        new_triangles = torch.stack([first_point, second_point, new_point], dim = 1) #[?, 3], edge here is flipped compared to edge in neighbouring triangle (so first_point is the second point in neighbouring edge)
        self.partial_delaunay_triangles.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3), new_triangles) #[batchsize, ntriangles + 2, 3]
        
        new_edge01 = neighbouring_edge_mask.view(self.batchsize, -1).nonzero(as_tuple = True)[1] #[?]
        
        """we are currently storing which triangles have to be inserted, via the edges along the perimeter of the delaunay cavity, we need to compute which edge is to the 'left'/'right' of each edge"""
        """don't have the memory to do a batchsize * n * n boolean search, don't have the speed to do a batchsize^2 search (as would be the case for sparse matrix or similar)"""
        """best alternative: rotate the edge around right point, repeat until hit edge in mask (will never go to an edge of a removed triangle before we hit edge in mask) should basically be order 1!!!!!"""
        
        neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?]
        next_neighbouring_edge_index = torch.empty_like(neighbouring_edge_index) #[?]
        
        rotating_flipped_neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?], initialise
        todo_mask = torch.ones_like(next_neighbouring_edge_index, dtype = torch.bool) #[?]
        while todo_mask.sum():
            rotating_neighbouring_edge_index = rotating_flipped_neighbouring_edge_index + 1 - 3 * (rotating_flipped_neighbouring_edge_index % 3 == 2) #[todo_mask.sum()], gets smaller until nothing left EFFICIENCY (this may be seriously stupid, as it requires making a bunch of copies when I could be doing stuff inplace)
            
            update_mask = neighbouring_edge_mask[rotating_neighbouring_edge_index] #[todo_mask.sum()]
            update_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(todo_mask, update_mask) #[?]
            
            next_neighbouring_edge_index.masked_scatter_(update_mask_unravel, rotating_neighbouring_edge_index.masked_select(update_mask)) #[?]
            
            todo_mask.masked_fill_(update_mask_unravel, False) #[?]
            rotating_flipped_neighbouring_edge_index = edges[rotating_neighbouring_edge_index.masked_select(update_mask.logical_not())] #[todo_mask.sum()]
        triangle_index = new_triangles_mask.view(-1).nonzero(as_tuple = True)[0] #[?], index goes up to batchsize * (ntriangles + 2), this is needed for when we invert the permutation by scattering (won't scatter same number of triangles per batch)
        
        next_triangle_index = torch.empty_like(edges).masked_scatter_(neighbouring_edge_mask, triangle_index)[next_neighbouring_edge_index] #[?], index goes up to batchsize * (ntriangles + 2)
        next_edge = 3 * next_triangle_index + 1 #[?]
        
        invert_permutation = torch.empty_like(new_triangles_mask.view(-1), dtype=torch.long) #[batchsize * (ntriangles + 2)]
        invert_permutation[next_triangle_index] = triangle_index #[batchsize * (ntriangles + 2)]
        previous_triangle_index = invert_permutation.masked_select(new_triangles_mask.view(-1)) #[?]
        previous_edge = 3 * previous_triangle_index + 2 #[?]
        
        """in the above we rotated around 'first_point' in our new triangles"""
        new_edge20 = next_edge % ((self.ntriangles + 2) * 3) #[?]
        new_edge12 = previous_edge % ((self.ntriangles + 2) * 3) #[?]
        
        new_edges = torch.stack([new_edge01, new_edge12, new_edge20], dim = 1) #[?, 3]
        self.partial_delaunay_edges.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3).reshape(self.batchsize, -1), new_edges) #[batchsize, (ntriangles + 2) * 3]
        
        self.ntriangles += 2
        """currently only count the extra triangles you replace (not the on you have to remove because you're located there, and not the ones you make because you have to create two more"""
        self.cost += (n_new_triangles - 3)
        self.points_mask.scatter_(1, point_index.unsqueeze(1).expand(-1, self.npoints), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 8, batchsize = 64, nsamples = 64):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(3, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= 0
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 0
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement_binary ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.4) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -(math.log(math.factorial(npoints - 3)) + 0.7)) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), env.logprob[0].item())

In [ ]:
train() #5 points lets GOOOO

4.5625 -4.526564598083496 -4.780689239501953 -4.606431007385254
3.875 -4.51965856552124 -4.777093887329102 -4.920797824859619
3.96875 -4.4756999015808105 -4.777926921844482 -4.776201248168945
3.8125 -4.408394813537598 -4.766851425170898 -4.830983638763428
3.90625 -4.29315185546875 -4.749675750732422 -4.510761260986328
4.0 -4.339272499084473 -4.756731033325195 -4.611299514770508
3.46875 -4.373560905456543 -4.763289451599121 -4.547156810760498
3.78125 -4.406455039978027 -4.770651817321777 -4.710372447967529
3.84375 -4.459534645080566 -4.776033401489258 -4.8399658203125
4.15625 -4.389991283416748 -4.767828464508057 -4.690505027770996
4.15625 -4.404461860656738 -4.769424915313721 -4.712284088134766
3.4375 -4.309226989746094 -4.751223087310791 -4.835796356201172
3.5625 -4.198622703552246 -4.707251071929932 -5.39250373840332
3.65625 -4.286349296569824 -4.747105598449707 -4.524640083312988
3.59375 -4.342093467712402 -4.756507873535156 -4.908664703369141
3.65625 -4.452999114990234 -4.770508289

3.625 -4.161331653594971 -4.706459045410156 -4.531759262084961
3.53125 -4.110810279846191 -4.694910526275635 -5.001943111419678
3.40625 -4.154590606689453 -4.709342956542969 -5.173217296600342
3.65625 -4.213791847229004 -4.731725215911865 -4.288405418395996
3.3125 -4.26734733581543 -4.738689422607422 -4.340150833129883
3.96875 -4.2947869300842285 -4.757785797119141 -5.048198699951172
3.96875 -4.261308670043945 -4.732351303100586 -4.97698974609375
3.96875 -4.224885940551758 -4.725635528564453 -4.955530166625977
3.71875 -4.115618705749512 -4.687260150909424 -4.089549541473389
3.375 -4.124932289123535 -4.683099269866943 -4.162351131439209
3.34375 -4.17689323425293 -4.727904796600342 -4.69888973236084
3.28125 -4.2516279220581055 -4.7389116287231445 -5.006295204162598
3.5 -4.356708526611328 -4.758423328399658 -4.820809364318848
3.59375 -4.339322090148926 -4.7655181884765625 -4.9599761962890625
3.5 -4.250361442565918 -4.742398738861084 -4.250362873077393
3.71875 -4.238726615905762 -4.7274961

3.84375 -4.161881446838379 -4.722352027893066 -4.99821138381958
3.28125 -4.195004940032959 -4.733510971069336 -5.045742511749268
3.5 -4.2298760414123535 -4.749475479125977 -4.971584796905518
3.25 -4.210083961486816 -4.744134902954102 -4.170979022979736
3.5625 -4.182257652282715 -4.7316694259643555 -4.841703414916992
3.25 -4.137479782104492 -4.725140571594238 -5.00700044631958
3.46875 -4.181623458862305 -4.721948146820068 -4.987795829772949
3.09375 -4.133786678314209 -4.7067952156066895 -4.442906379699707
3.84375 -4.196107864379883 -4.749152183532715 -4.942604064941406
3.40625 -4.226432800292969 -4.740716934204102 -4.272526264190674
3.40625 -4.2349090576171875 -4.753156661987305 -5.017332077026367
3.0 -4.182187080383301 -4.727146148681641 -5.014593601226807
3.71875 -4.152986526489258 -4.735307693481445 -4.931520462036133
3.21875 -4.145264148712158 -4.7171711921691895 -5.025282382965088
3.5 -4.110965728759766 -4.731398105621338 -4.19496488571167
3.625 -4.074061393737793 -4.73202657699585

3.1875 -3.175844669342041 -4.642988204956055 -5.289035797119141
3.3125 -3.0202057361602783 -4.633723258972168 -5.312630653381348
3.0 -3.0837674140930176 -4.610688209533691 -5.449031829833984
3.125 -2.737889528274536 -4.555574893951416 -4.981758117675781
3.15625 -2.985177993774414 -4.575560569763184 -5.099598407745361
3.25 -3.0901174545288086 -4.628908634185791 -4.212570667266846
3.09375 -3.2328829765319824 -4.648282527923584 -4.750833511352539
2.78125 -3.4031383991241455 -4.665295600891113 -4.365910530090332
3.28125 -3.4399120807647705 -4.680828094482422 -3.3994741439819336
3.1875 -3.379213809967041 -4.675582408905029 -4.9815568923950195
3.46875 -3.2520740032196045 -4.655461311340332 -5.13089656829834
3.34375 -3.0876832008361816 -4.631921291351318 -5.156937122344971
2.8125 -3.2252554893493652 -4.6289472579956055 -4.971164226531982
3.21875 -3.0053279399871826 -4.595360279083252 -4.9735307693481445
3.28125 -2.9343369007110596 -4.566335678100586 -5.022749423980713
2.84375 -3.1113255023956

2.875 -2.9698965549468994 -4.599484443664551 -4.998876571655273
3.25 -2.8453493118286133 -4.552055835723877 -3.5794196128845215
2.9375 -3.046475648880005 -4.6219282150268555 -4.565228462219238
3.15625 -3.05110502243042 -4.618935585021973 -2.598701238632202
2.75 -2.885371208190918 -4.5989603996276855 -4.807327747344971
2.96875 -3.0338189601898193 -4.639311790466309 -3.4908952713012695
3.15625 -2.8475019931793213 -4.6082000732421875 -2.7418270111083984
2.96875 -2.7908449172973633 -4.556489944458008 -4.955669403076172
3.21875 -2.8423078060150146 -4.554816722869873 -4.885436534881592
3.25 -2.9684324264526367 -4.598339080810547 -3.4405694007873535
3.21875 -2.912625312805176 -4.6107611656188965 -4.945770263671875
2.90625 -2.975259780883789 -4.5990424156188965 -4.94563627243042
3.21875 -3.108384132385254 -4.647585868835449 -3.2206223011016846
3.09375 -2.994288444519043 -4.603930950164795 -4.834070205688477
2.96875 -2.9332363605499268 -4.63370418548584 -4.909098148345947
3.3125 -2.898484706878

...

3.03125 -2.112192392349243 -4.500120162963867 -4.795300006866455
2.84375 -2.1268157958984375 -4.481040000915527 -4.595643997192383
2.78125 -2.1379618644714355 -4.517313003540039 -4.9925360679626465
3.0625 -2.1975209712982178 -4.517640113830566 -5.036247730255127
3.1875 -2.108511447906494 -4.509954929351807 -4.996669292449951
3.15625 -2.0923564434051514 -4.441813945770264 -5.114401817321777
3.03125 -2.088644027709961 -4.529480457305908 -1.7151769399642944
3.40625 -1.9810450077056885 -4.441990852355957 -4.831707954406738
2.71875 -2.1419129371643066 -4.483596324920654 -5.3326520919799805
2.96875 -2.0666069984436035 -4.465085029602051 -4.897765636444092
3.0625 -1.8389310836791992 -4.489248275756836 -4.880932807922363
2.96875 -2.240475654602051 -4.463167190551758 -4.87255859375
2.9375 -1.9967796802520752 -4.483684539794922 -2.3133132457733154
2.84375 -2.042175531387329 -4.453296661376953 -4.753914833068848
3.03125 -2.1595001220703125 -4.501142501831055 -4.916809558868408
2.875 -2.1394586563

2.8125 -2.049424648284912 -4.455002784729004 -4.97220516204834
3.0 -1.9416135549545288 -4.510317802429199 -4.8957905769348145
3.09375 -1.938412070274353 -4.431910991668701 -5.212467193603516
2.90625 -2.1254546642303467 -4.453421592712402 -5.0233917236328125
2.9375 -1.9994455575942993 -4.46953821182251 -4.765594482421875
3.3125 -2.002073287963867 -4.512333393096924 -5.089792728424072
3.03125 -2.1441593170166016 -4.4678568840026855 -1.753888487815857
3.03125 -2.0755014419555664 -4.465756416320801 -5.14603328704834
3.03125 -1.9053173065185547 -4.4607062339782715 -5.1869354248046875
3.125 -2.153380870819092 -4.4801177978515625 -1.7847604751586914
3.0 -2.0584096908569336 -4.5114216804504395 -2.5611257553100586
3.0 -1.9798083305358887 -4.414449214935303 -4.966829776763916
3.1875 -2.225938320159912 -4.4551100730896 -1.8854752779006958
3.09375 -2.1573712825775146 -4.434138298034668 -5.065609455108643
3.09375 -2.242246150970459 -4.478203773498535 -5.00739860534668
3.21875 -2.1789557933807373 -4

2.71875 -2.09421443939209 -4.49440336227417 -4.932727813720703
2.96875 -2.0770182609558105 -4.486331462860107 -5.24274206161499
2.71875 -2.2581701278686523 -4.540904521942139 -4.965386390686035
3.3125 -2.402657985687256 -4.529767036437988 -4.509191989898682
2.84375 -2.015296459197998 -4.472424030303955 -4.909348487854004
3.125 -2.066335916519165 -4.452341556549072 -2.296125888824463
2.84375 -2.0241079330444336 -4.509181499481201 -2.0095324516296387
2.90625 -1.9282902479171753 -4.463180065155029 -5.057862281799316
2.90625 -2.090740203857422 -4.492722511291504 -4.921987056732178
2.84375 -2.0874338150024414 -4.478144645690918 -1.7780325412750244
2.75 -2.270367383956909 -4.455437660217285 -5.027369022369385
3.03125 -2.149007797241211 -4.465176582336426 -4.946913719177246
3.15625 -1.9537477493286133 -4.475422382354736 -4.924984455108643
2.8125 -1.9650745391845703 -4.489778518676758 -1.734481930732727
3.0625 -2.130089044570923 -4.526172637939453 -3.147707462310791
3.03125 -2.0808019638061523

2.625 -2.006251335144043 -4.409443378448486 -2.6486616134643555
2.96875 -2.010770320892334 -4.469017505645752 -1.5871245861053467
2.625 -2.2175517082214355 -4.498937606811523 -4.9613542556762695
3.125 -1.9726364612579346 -4.425710678100586 -4.815932750701904
2.96875 -2.323610544204712 -4.440155029296875 -4.928348064422607
2.875 -2.147930860519409 -4.4994587898254395 -5.040389537811279
3.09375 -2.053053617477417 -4.456168174743652 -5.030309677124023
2.8125 -1.9994759559631348 -4.447649002075195 -5.039580345153809
3.0625 -2.0476951599121094 -4.464076519012451 -5.323116779327393
2.84375 -2.109856128692627 -4.461037635803223 -5.081345081329346
3.0 -2.032670259475708 -4.4235053062438965 -4.84879732131958
3.1875 -1.9186291694641113 -4.358826160430908 -5.197985649108887
3.15625 -2.0009806156158447 -4.3982062339782715 -4.913077354431152
2.875 -2.081960678100586 -4.45328950881958 -4.419560432434082
3.0625 -2.089782238006592 -4.382154941558838 -5.097673416137695
2.71875 -1.943137764930725 -4.397

3.03125 -2.1220290660858154 -4.432079792022705 -5.019007205963135
3.28125 -2.055116653442383 -4.4318389892578125 -4.990675449371338
3.09375 -2.005837917327881 -4.434808731079102 -4.917500019073486
2.8125 -1.885136604309082 -4.397047519683838 -1.5147844552993774
2.96875 -1.8573718070983887 -4.370917320251465 -5.011394023895264
3.375 -2.1485068798065186 -4.494871139526367 -1.7215874195098877
2.9375 -2.0372939109802246 -4.418419361114502 -5.1135053634643555
2.9375 -2.047926664352417 -4.501547336578369 -4.302083492279053
3.0625 -2.1722664833068848 -4.540867328643799 -3.639759063720703
3.03125 -2.230085611343384 -4.5357666015625 -4.986672401428223
3.25 -2.274733304977417 -4.504903316497803 -2.379330635070801
3.125 -1.9519790410995483 -4.44845724105835 -4.776403903961182
3.21875 -2.1326708793640137 -4.4730119705200195 -1.6417838335037231
2.90625 -2.148402690887451 -4.421512126922607 -5.134409427642822
2.8125 -2.0039093494415283 -4.449939727783203 -5.136726379394531
3.03125 -2.04350209236145 

In [ ]:
train() #changes some stuff

4.265625 -4.666250705718994 -4.784326076507568 -4.737793922424316
3.46875 -4.404719829559326 -4.768851280212402 -4.907869338989258
3.609375 -4.024624347686768 -4.659417152404785 -4.308406829833984
3.796875 -3.5761592388153076 -4.463003635406494 -3.5155858993530273
3.8125 -3.807979106903076 -4.491922855377197 -3.709899425506592
3.4375 -3.5535120964050293 -4.307840347290039 -3.7096457481384277
3.46875 -3.336097002029419 -4.243531703948975 -3.888633966445923
3.515625 -3.201611042022705 -4.176779270172119 -5.952202796936035
3.171875 -3.2709169387817383 -4.236358165740967 -4.215490341186523
3.4375 -3.263731002807617 -4.208045959472656 -3.836066722869873
3.140625 -3.4100143909454346 -4.268549919128418 -4.277129173278809
3.140625 -3.295368194580078 -4.307026386260986 -3.877622604370117
3.328125 -3.3189949989318848 -4.343939304351807 -4.552635669708252
2.875 -3.2950119972229004 -4.416830539703369 -4.983826160430908
3.25 -3.2657008171081543 -4.397335529327393 -3.3344860076904297
2.96875 -3.1533

2.984375 -0.8673115968704224 -1.587491750717163 -1.6954772472381592
2.78125 -0.7825500965118408 -1.3895167112350464 -1.1551594734191895
3.0 -0.8052932024002075 -1.507299542427063 -0.8449438810348511
2.75 -0.8642609715461731 -1.5713709592819214 -1.5336506366729736
3.03125 -0.7805637121200562 -1.5144366025924683 -0.7918276786804199
2.78125 -0.8861854076385498 -1.5772428512573242 -1.5473079681396484
3.03125 -0.7685301303863525 -1.4287582635879517 -0.4220534861087799
2.78125 -0.6564217805862427 -1.3032392263412476 -0.1345372200012207
2.890625 -0.7108314633369446 -1.3869390487670898 -0.9124069213867188
3.015625 -0.6961888074874878 -1.3499356508255005 -1.1507782936096191
2.765625 -0.7888134717941284 -1.4465504884719849 -2.179105758666992
3.109375 -0.6795477867126465 -1.3491781949996948 -1.2320244312286377
2.828125 -0.7815878391265869 -1.408504843711853 -0.2532346248626709
2.765625 -0.7712303400039673 -1.4257506132125854 -0.11350548267364502
2.609375 -0.707872748374939 -1.3078975677490234 -0.

2.984375 -0.38958457112312317 -0.7114250063896179 -0.12641334533691406
3.03125 -0.4342014491558075 -0.8386820554733276 -0.2536754608154297
3.0625 -0.45791155099868774 -0.8410717844963074 -0.04994344711303711
2.6875 -0.40823686122894287 -0.7799661159515381 -0.12039732933044434
2.953125 -0.48960810899734497 -0.8561857342720032 -0.18785560131072998
2.96875 -0.484056681394577 -0.8882826566696167 -4.014469623565674
2.890625 -0.447336345911026 -0.8324374556541443 -0.2989921569824219
2.921875 -0.5039517879486084 -0.8499383330345154 -1.7626852989196777
2.953125 -0.5615473389625549 -1.0245436429977417 -3.9287514686584473
2.6875 -0.40505558252334595 -0.871789276599884 -1.3596348762512207
2.9375 -0.5148621797561646 -0.981382429599762 -0.39562344551086426
2.890625 -0.5164042711257935 -1.0049275159835815 -0.2568168640136719
2.859375 -0.4551026523113251 -0.9254158139228821 -0.7875062227249146
2.859375 -0.45249101519584656 -0.8990034461021423 -0.26144516468048096
3.046875 -0.5126873254776001 -0.98551

2.984375 -0.5848985314369202 -1.0896821022033691 -2.7404675483703613
2.765625 -0.5078327655792236 -0.9371477961540222 -0.5113399028778076
2.84375 -0.4757864773273468 -0.8923946022987366 -0.5554364919662476
2.859375 -0.4207128882408142 -0.8755402565002441 -1.4666051864624023
2.921875 -0.5046344995498657 -0.9359294772148132 -4.632122993469238
3.171875 -0.4372791349887848 -0.8173050880432129 -0.5521345138549805
2.890625 -0.4012106657028198 -0.8269044756889343 -3.0549087524414062
3.109375 -0.5737155675888062 -0.965122640132904 -1.312084674835205
3.09375 -0.48733094334602356 -0.8636718392372131 -2.4083147048950195
2.828125 -0.4293195605278015 -0.8376810550689697 -0.6378915309906006
2.75 -0.44197878241539 -0.8186242580413818 -1.4220378398895264
3.078125 -0.452992707490921 -0.8684441447257996 -0.3955991268157959
2.703125 -0.4547920227050781 -0.846004843711853 -0.04470491409301758
2.59375 -0.3766244649887085 -0.7395026087760925 -2.441598415374756
2.984375 -0.354516863822937 -0.7132667899131775

2.796875 -0.2562906742095947 -0.5031474232673645 -0.020905494689941406
2.796875 -0.24650831520557404 -0.4859057366847992 -0.0040836334228515625
2.71875 -0.32134464383125305 -0.5959809422492981 -1.0243110656738281
2.90625 -0.2597278654575348 -0.5429480671882629 -0.7418866157531738
2.6875 -0.29319700598716736 -0.5674561858177185 -0.002673625946044922
3.171875 -0.21925723552703857 -0.43788689374923706 -0.0009212493896484375
2.875 -0.39010265469551086 -0.6789370775222778 -0.6604738235473633
2.859375 -0.32497838139533997 -0.6350268721580505 -0.5521259307861328
2.828125 -0.24632732570171356 -0.5170507431030273 -0.5388317108154297
2.640625 -0.23515671491622925 -0.5066322684288025 -0.06493949890136719
3.046875 -0.2826419472694397 -0.6106690168380737 -0.18597984313964844
2.875 -0.2855783700942993 -0.5494744777679443 -1.4268282651901245
2.890625 -0.3175637125968933 -0.6252418756484985 -0.03332805633544922
2.984375 -0.4049879312515259 -0.7548744678497314 -0.16919803619384766
2.8125 -0.24091726541

...

2.6875 -0.18273931741714478 -0.36433467268943787 -0.2823648452758789
2.78125 -0.147897869348526 -0.3102279603481293 -3.9748659133911133
2.734375 -0.18181169033050537 -0.357438862323761 -1.0257759094238281
2.5625 -0.2445388287305832 -0.4440390169620514 -0.6104153394699097
2.953125 -0.1562945544719696 -0.3348080515861511 -0.010848522186279297
2.671875 -0.19472762942314148 -0.426170289516449 -0.000461578369140625
2.640625 -0.17170585691928864 -0.3149051368236542 -1.9073486328125e-06
2.671875 -0.19848519563674927 -0.39814385771751404 -0.0025377273559570312
2.828125 -0.2152681052684784 -0.4247221350669861 -0.5821471214294434
2.84375 -0.172738179564476 -0.32619187235832214 -0.029677391052246094
2.625 -0.1991194188594818 -0.3806082308292389 -2.2039809226989746
2.9375 -0.19894066452980042 -0.3801968991756439 -0.0048065185546875
2.53125 -0.16509085893630981 -0.3219395577907562 -0.012912750244140625
2.84375 -0.20019404590129852 -0.39303794503211975 -0.15613842010498047
2.828125 -0.16058662533760

2.765625 -0.21220409870147705 -0.40347912907600403 -1.9073486328125e-06
3.03125 -0.1875312775373459 -0.41824617981910706 -0.0007253885269165039
2.9375 -0.17616303265094757 -0.34278398752212524 -0.565106987953186
2.9375 -0.16701087355613708 -0.34539684653282166 -0.011812686920166016
2.65625 -0.18578635156154633 -0.36299431324005127 -0.003532886505126953
2.84375 -0.1797809898853302 -0.35491326451301575 -0.0885171890258789
2.953125 -0.1198011264204979 -0.2669377624988556 -0.005179882049560547
2.90625 -0.19428949058055878 -0.39785364270210266 -1.6020326614379883
2.890625 -0.10683922469615936 -0.22785376012325287 -0.0039958953857421875
2.921875 -0.15763068199157715 -0.33770522475242615 -0.08580398559570312
2.625 -0.1287004053592682 -0.2562543749809265 -0.000640869140625
2.890625 -0.16973057389259338 -0.359201043844223 -1.0990095138549805
2.859375 -0.17599892616271973 -0.3433349132537842 -2.47955322265625e-05
2.828125 -0.14553286135196686 -0.29323628544807434 -0.01786494255065918
2.75 -0.212

2.75 -0.1434205174446106 -0.28080594539642334 -0.44828033447265625
2.90625 -0.17413923144340515 -0.3090713620185852 -0.8665480613708496
2.84375 -0.16154515743255615 -0.31312817335128784 -0.9924945831298828
2.984375 -0.1845175176858902 -0.3718588948249817 -0.30991601943969727
2.796875 -0.19135500490665436 -0.32835081219673157 -0.050266265869140625
2.90625 -0.16479560732841492 -0.3500310182571411 -0.5501968860626221
2.859375 -0.15855640172958374 -0.34339040517807007 0.0
2.859375 -0.15792617201805115 -0.3177106976509094 -3.4332275390625e-05
2.984375 -0.1297668218612671 -0.26664620637893677 -0.00022268295288085938
2.734375 -0.19249966740608215 -0.3803500831127167 -0.009343147277832031
2.6875 -0.1643434315919876 -0.3004487156867981 -0.6873149871826172
2.671875 -0.19046983122825623 -0.34081995487213135 -0.7370465397834778
2.765625 -0.11226978898048401 -0.2510886788368225 -4.673004150390625e-05
2.859375 -0.16633546352386475 -0.2889409363269806 -1.1491737365722656
2.859375 -0.17717276513576508

2.578125 -0.17209705710411072 -0.3137522339820862 -0.0005550384521484375
2.734375 -0.15611591935157776 -0.31437504291534424 -0.0013179779052734375
2.703125 -0.22785484790802002 -0.440414696931839 -0.0009832382202148438
2.6875 -0.1395924687385559 -0.2934733033180237 -0.007450103759765625
2.640625 -0.10478626191616058 -0.2333107590675354 -0.1882786750793457
2.984375 -0.1996571570634842 -0.37374892830848694 -0.25679874420166016
2.90625 -0.16109594702720642 -0.31292012333869934 -0.3516106605529785
2.6875 -0.14787206053733826 -0.29172033071517944 -0.06072807312011719
2.5625 -0.16318929195404053 -0.3467176854610443 -0.09981536865234375
2.90625 -0.19199666380882263 -0.3772551715373993 -1.4781951904296875e-05
2.5 -0.15143254399299622 -0.300968736410141 -0.09768658876419067
2.796875 -0.1606944501399994 -0.32262957096099854 -1.0957956314086914
2.625 -0.12129849940538406 -0.2606293857097626 -0.00626373291015625
2.796875 -0.18914169073104858 -0.348227858543396 -0.02639329433441162
2.765625 -0.0929

2.796875 -0.15062303841114044 -0.3035213053226471 -4.291534423828125e-05
2.625 -0.21424371004104614 -0.3780237138271332 -0.4078216552734375
2.71875 -0.19175928831100464 -0.3496912121772766 -0.21734333038330078
2.65625 -0.17152561247348785 -0.35324814915657043 -0.026397705078125
2.828125 -0.2034417688846588 -0.35879266262054443 -1.71661376953125e-05
2.96875 -0.19510643184185028 -0.36665070056915283 -0.41242074966430664
2.65625 -0.16336838901042938 -0.321551650762558 -0.08551406860351562
2.84375 -0.15251249074935913 -0.2686794698238373 -0.0007038116455078125
2.796875 -0.1294923722743988 -0.22694800794124603 -0.1563803255558014
2.75 -0.14470934867858887 -0.2808488607406616 -0.36763763427734375
2.59375 -0.1326759159564972 -0.2571389079093933 -0.06194591522216797
2.609375 -0.14349856972694397 -0.27999797463417053 -1.6732864379882812
2.796875 -0.1739245057106018 -0.3050447702407837 -0.0001125335693359375
2.671875 -0.15335114300251007 -0.2577740252017975 -1.620545744895935
2.453125 -0.1396281

In [ ]:
train() #changed some more stuff

3.8125 -4.463106632232666 -4.764679908752441 -4.816375255584717
3.609375 -4.004912376403809 -4.6780104637146 -4.270087718963623
3.75 -3.952754259109497 -4.640414714813232 -4.448251247406006
3.796875 -3.7114150524139404 -4.581544399261475 -4.080312252044678
3.53125 -3.3862464427948 -4.4334001541137695 -4.8271026611328125
3.828125 -3.4160218238830566 -4.366032123565674 -4.020120143890381
3.4375 -3.402089834213257 -4.395946025848389 -5.211293697357178
3.59375 -3.4131407737731934 -4.3454813957214355 -4.451146125793457
3.71875 -3.3501906394958496 -4.262554168701172 -3.7342190742492676
3.609375 -3.4051780700683594 -4.296189308166504 -3.5001442432403564
3.96875 -3.5381579399108887 -4.423122406005859 -4.131859302520752
3.65625 -3.7814581394195557 -4.5129570960998535 -4.442102432250977
3.21875 -3.7932777404785156 -4.573587894439697 -5.030518531799316
3.5625 -3.8386974334716797 -4.597740650177002 -4.490254878997803
3.171875 -3.565917730331421 -4.5616068840026855 -4.1458821296691895
3.5625 -3.467

3.328125 -0.8265183568000793 -1.6000417470932007 -0.8265219330787659
3.203125 -0.9586758613586426 -1.6894047260284424 -1.4342950582504272
3.203125 -0.8880060911178589 -1.5809929370880127 -2.469602584838867
3.21875 -0.9971432089805603 -1.7496744394302368 -0.9169373512268066
2.859375 -0.9412356019020081 -1.7543830871582031 -1.1348717212677002
2.984375 -0.7919695377349854 -1.6223410367965698 -0.7099497318267822
2.921875 -0.90272057056427 -1.7476067543029785 -0.5009048581123352
2.984375 -1.0405280590057373 -1.8347618579864502 -1.2566277980804443
3.125 -0.9887428283691406 -1.7490646839141846 -0.6128703355789185
3.203125 -1.033776879310608 -1.8283133506774902 -0.6931993365287781
2.953125 -0.9809936285018921 -1.693411111831665 -2.7366397380828857
2.90625 -0.9575910568237305 -1.6527419090270996 -0.8025341033935547
3.078125 -0.9069012403488159 -1.4801136255264282 -0.9525585174560547
2.734375 -0.6902380585670471 -1.3658332824707031 -3.3489038944244385
3.015625 -0.6795383095741272 -1.304043292999

3.046875 -0.9610321521759033 -1.5028119087219238 -0.979485034942627
2.765625 -0.6606109738349915 -1.4068900346755981 -0.19044379889965057
2.984375 -0.6388919353485107 -1.4006497859954834 -0.00637507438659668
3.0625 -0.6962512731552124 -1.355868935585022 -2.4761109352111816
2.734375 -0.6863964796066284 -1.3242367506027222 -1.0987458229064941
3.0 -0.8796480894088745 -1.4363528490066528 -0.5395830869674683
3.078125 -0.9306210279464722 -1.4944480657577515 -1.150247573852539
2.890625 -0.823885440826416 -1.423507809638977 -0.99430912733078
3.171875 -0.9177089929580688 -1.4135977029800415 -0.13269400596618652
2.9375 -0.9226846694946289 -1.4547953605651855 -0.4229433536529541
3.15625 -0.9651933312416077 -1.4786978960037231 -2.6626944541931152
2.84375 -0.812563955783844 -1.425429344177246 -1.4679388999938965
2.875 -0.8017880916595459 -1.4824546575546265 -1.1217530965805054
2.859375 -0.7982191443443298 -1.3685945272445679 -0.1575092077255249
3.109375 -0.8015519380569458 -1.3840457201004028 -0.71

3.171875 -0.9755971431732178 -1.7328740358352661 -0.23063784837722778
2.9375 -1.0430476665496826 -1.5805460214614868 -0.603742241859436
2.84375 -1.0511975288391113 -1.6496602296829224 -0.1287994384765625
2.890625 -1.0562095642089844 -1.7289332151412964 -0.9660288095474243
3.046875 -1.0037949085235596 -1.6354585886001587 -1.0761367082595825
3.015625 -0.958808422088623 -1.716944694519043 -1.5825629234313965
3.109375 -1.659013271331787 -1.7313001155853271 -0.07771587371826172
2.96875 -1.5046426057815552 -1.7784773111343384 -0.9353370666503906
3.046875 -1.1902974843978882 -1.8959413766860962 -0.1331859827041626
2.859375 -1.063099980354309 -1.8969863653182983 -0.06751608848571777
2.890625 -1.3160929679870605 -1.8649481534957886 -0.5338101387023926
2.703125 -1.0570012331008911 -1.7757534980773926 -0.102150559425354
2.921875 -1.285739541053772 -1.8939841985702515 -0.6778960227966309
2.859375 -1.4512280225753784 -2.0513417720794678 -4.466838836669922
3.03125 -1.7361125946044922 -2.145779132843

2.953125 -1.5123580694198608 -1.9887281656265259 -2.8301193714141846
2.796875 -1.3600270748138428 -1.7963371276855469 -0.9895534515380859
2.875 -1.3932417631149292 -1.7297559976577759 -0.5589824914932251
2.84375 -1.2070956230163574 -1.845151662826538 -0.808722972869873
3.0 -1.407968521118164 -1.8774734735488892 -1.031433343887329
3.015625 -1.023325800895691 -1.7953358888626099 -1.797945499420166
2.859375 -1.2696762084960938 -1.8767586946487427 -0.3893277645111084
2.9375 -1.3263823986053467 -1.9763907194137573 -1.164692759513855
2.984375 -1.077985405921936 -1.8753013610839844 -0.11772739887237549
3.078125 -0.8173925876617432 -1.6999043226242065 -2.5627551078796387
3.03125 -0.9274234771728516 -1.7549721002578735 -1.5401630401611328
2.78125 -1.0889124870300293 -1.67892324924469 -0.291640043258667
3.015625 -1.0223991870880127 -1.58331298828125 -0.33607304096221924
3.09375 -1.280890703201294 -1.7236336469650269 -1.1153130531311035
2.84375 -0.7955437898635864 -1.579953670501709 -0.2025148868

...

2.84375 -3.365424156188965 -4.226811408996582 -3.3034980297088623
2.671875 -2.892897605895996 -4.306647777557373 -2.1792666912078857
2.6875 -3.0179874897003174 -4.1706013679504395 -0.9411730766296387
2.75 -3.075624704360962 -4.306703090667725 -1.5178518295288086
2.65625 -3.080909252166748 -4.2227702140808105 -3.0098938941955566
2.53125 -3.150698661804199 -4.310649871826172 -2.65971302986145
2.5 -2.982055187225342 -4.3396711349487305 -2.9872772693634033
2.65625 -3.7491211891174316 -4.287180423736572 -2.296135425567627
2.578125 -2.969205856323242 -4.303484916687012 -3.858719825744629
2.578125 -3.120708465576172 -4.178544521331787 -0.980134129524231
2.78125 -3.2342607975006104 -4.281601905822754 -4.476643085479736
2.40625 -3.013031005859375 -4.247086524963379 -0.9503105878829956
2.578125 -3.475876808166504 -4.2236647605896 -1.626456618309021
2.609375 -3.0410196781158447 -4.23600959777832 -0.8357819318771362
2.625 -3.0475239753723145 -4.399652481079102 -3.9696216583251953
2.4375 -2.9769444

2.359375 -3.1204588413238525 -4.2844014167785645 -4.3429670333862305
2.609375 -3.476066827774048 -4.255980491638184 -3.9727466106414795
2.65625 -3.4244539737701416 -4.4173808097839355 -2.433568000793457
2.515625 -3.3743019104003906 -4.418801307678223 -2.0535359382629395
2.5 -2.9256954193115234 -4.360010623931885 -3.682603359222412
2.46875 -2.9640753269195557 -4.332922458648682 -2.5074121952056885
2.640625 -3.4678218364715576 -4.415643215179443 -4.974316596984863
2.6875 -3.354884386062622 -4.419375896453857 -3.6224915981292725
2.5625 -2.624675750732422 -4.302868366241455 -1.8660603761672974
2.6875 -3.5016584396362305 -4.413967609405518 -1.5687919855117798
2.640625 -3.2876815795898438 -4.284039497375488 -1.7506043910980225
2.484375 -2.695249557495117 -4.2647833824157715 -2.6258950233459473
2.375 -2.6698179244995117 -4.127103805541992 -2.841073989868164
2.265625 -2.8813438415527344 -4.166773796081543 -2.9724819660186768
2.765625 -3.3410940170288086 -4.274855136871338 -0.25059542059898376


2.59375 -3.8856160640716553 -4.168659687042236 -4.48344087600708
2.578125 -3.083974838256836 -4.183183670043945 -1.2146081924438477
2.6875 -3.5518455505371094 -4.176572322845459 -1.3503470420837402
2.453125 -2.8521366119384766 -4.35610818862915 -3.2598116397857666
2.578125 -2.829775094985962 -4.1731038093566895 -0.45359086990356445
2.484375 -3.2743940353393555 -4.194493770599365 -2.591240644454956
2.390625 -2.8096656799316406 -4.101481914520264 -1.6957145929336548
2.34375 -2.8593387603759766 -4.142757415771484 -4.304574489593506
2.53125 -3.0186209678649902 -4.282934665679932 -5.668118953704834
2.4375 -2.845374584197998 -4.184564113616943 -2.9369935989379883
2.609375 -3.0904130935668945 -4.327215671539307 -2.1250360012054443
2.78125 -2.7069973945617676 -4.2547760009765625 -2.6427881717681885
2.421875 -2.9993784427642822 -4.203046798706055 -6.472596168518066
2.390625 -3.058678150177002 -3.9981749057769775 -4.935990333557129
2.40625 -2.9017531871795654 -4.219779014587402 -4.48092842102050

2.53125 -3.0233256816864014 -4.286876201629639 -1.2836828231811523
2.53125 -3.0290753841400146 -4.228699207305908 -0.5999748706817627
2.6875 -3.2288990020751953 -4.240839004516602 -1.3271583318710327
2.328125 -2.6281442642211914 -4.221926689147949 -0.062278032302856445
2.5625 -3.130960464477539 -4.200662612915039 -4.09812068939209
2.53125 -3.1257314682006836 -4.2560553550720215 -2.915452003479004
2.734375 -2.989023447036743 -4.3241190910339355 -1.081530213356018
2.59375 -3.3062400817871094 -4.251977443695068 -5.024487495422363
2.421875 -2.7007827758789062 -4.1672868728637695 -0.2438514232635498
2.625 -3.077326774597168 -4.180263042449951 -1.5017321109771729
2.359375 -2.8425018787384033 -4.204961776733398 -1.6922786235809326
2.46875 -2.7550816535949707 -4.23496675491333 -0.8222079873085022
2.640625 -3.3162291049957275 -4.174345970153809 -1.1408166885375977
2.46875 -2.7008676528930664 -4.144265651702881 -1.7489535808563232
2.34375 -2.7066688537597656 -4.1633195877075195 -1.42895293235778

2.59375 -3.0120277404785156 -4.1941094398498535 -3.144624710083008
2.5 -2.9642887115478516 -4.374571800231934 -4.556893825531006
2.46875 -2.691776752471924 -4.175868511199951 -1.1705832481384277
2.609375 -3.143538236618042 -4.192150592803955 -1.819413661956787
2.453125 -2.753514289855957 -4.208682060241699 -4.557279586791992
2.28125 -2.3769147396087646 -4.227652549743652 -5.11715030670166
2.53125 -3.110348701477051 -4.2117533683776855 -4.162535190582275
2.59375 -3.225921392440796 -4.202112674713135 -3.382817268371582
2.65625 -3.078838348388672 -4.234284400939941 -0.815717339515686
2.4375 -3.1799051761627197 -4.247844696044922 -2.577305793762207
2.453125 -2.7710888385772705 -4.1982831954956055 -3.647726535797119
2.578125 -3.323529005050659 -4.3390631675720215 -5.655422687530518
2.3125 -2.7141151428222656 -4.312557697296143 -2.1325671672821045
2.40625 -2.907078742980957 -4.243793964385986 -1.2193251848220825
2.53125 -2.5724997520446777 -4.102784633636475 -4.895237445831299
2.28125 -2.576